# HW3 - Social Network Analysis

## Import modules

In [1]:
import math
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
from networkx.drawing.nx_agraph import graphviz_layout
from tqdm import tqdm

## Useful functions

In [2]:
k=10

def print_dict(d, k=10):
    for item in list(d.items())[:k]:
        print(item)
        
def lists_overlap(l1, l2):
    return bool(set(l1) & set(l2))

## Load the data

In [3]:
casts = pd.read_csv('./../data/casts.csv', error_bad_lines = False, sep=';')

## Convert „casts“ data to a graph

In [4]:
G = nx.Graph()

# Print casts CSV column headers
print(casts.columns.tolist())

# Group movies by actors
movies_by_actor = casts.groupby('actor')['movie']
# movies_by_actor = casts[casts['actor_type'] < 'AA90'].groupby('actor')['movie']
movies_by_actor_dict = movies_by_actor.apply(list).to_dict()
# print(len(movies_by_actor_dict))
# print(movies_by_actor_dict['A.E. Matthews'])
# print(movies_by_actor_dict)

# Filter the supporting actor
if 's a' in movies_by_actor_dict:
    del movies_by_actor_dict['s a']

# Filter numeric actor values
# Using dictionary comprehension to find list
# Get numeric actors
delete = [key for key in movies_by_actor_dict if key.isnumeric()]

# Delete the key
for key in delete: del movies_by_actor_dict[key]
        
# Create the graph
for actor in movies_by_actor_dict.keys():
    G.add_node(actor)
        
# Test if two lists overlaps
# print(bool(set(movies_by_actor_dict['A.E. Matthews']) & set(movies_by_actor_dict['David Tree'])))
# print(bool(set(movies_by_actor_dict['A.E. Matthews']) & set(movies_by_actor_dict['Athene Seyler'])))

for actor1, movies1 in tqdm(movies_by_actor_dict.items(), total=len(G.nodes())):
    for actor2, movies2 in movies_by_actor_dict.items():
        if actor1 != actor2 and lists_overlap(movies1, movies2):
            G.add_edge(actor1, actor2)

['actor_type', 'movie', 'actor', 'role_type', 'role']


100%|██████████| 16610/16610 [05:17<00:00, 52.31it/s]


## Dataset general statistics

In [5]:
def general_statistics(graph, visualize_components=False):
    n = len(graph.nodes())
    e = len(graph.edges())
    print('Number of nodes: ', n)
    print('Number of edges: ', e)
    print('Density: ', e / (n*(n-1)/2))
    components = list(nx.connected_components(graph))
    if visualize_components:
        pos = graphviz_layout(graph)
        nx.draw(graph, pos, with_labels=False, node_size=10)
    print('Number of components: ', len(components))
    
general_statistics(G)

Number of nodes:  16610
Number of edges:  152251
Density:  0.0011037660504019404
Number of components:  748


## Centralities

In [6]:
def compute_centralities(graph, centralities, k):
    for centrality in tqdm(centralities, total=len(centralities)):
        print('Top {} players by {}:'.format(k, centrality.__name__))
        c_dict = centrality(graph)
        c_dict = dict(sorted(c_dict.items(), key=lambda item: item[1], reverse=True))
        for item in list(c_dict.items()):
            graph.nodes[item[0]][centrality.__name__] = item[1]
        print_dict(c_dict)
        print('\n')
    return graph

G = compute_centralities(G, [nx.degree_centrality, nx.closeness_centrality, nx.betweenness_centrality, nx.eigenvector_centrality], k)
# G = compute_centralities(G, [nx.degree_centrality], k)
# print(G.nodes['Humphrey Bogart'])

 25%|██▌       | 1/4 [00:00<00:00,  7.93it/s]

Top 10 players by degree_centrality:
('Humphrey Bogart', 0.02582936961888133)
('James Stewart', 0.022457703654645073)
('Gary Cooper', 0.02221687037148534)
('John Gielgud', 0.02221687037148534)
('John Carradine', 0.02203624540911554)
('Peter Lorre', 0.021614787163586006)
('C.Aubrey Smith', 0.020229995785417544)
('Henry Fonda', 0.019447287615148412)
('Burt Lancaster', 0.018724787765669215)
('Basil Rathbone', 0.018604371124089348)


Top 10 players by closeness_centrality:


 50%|█████     | 2/4 [10:00<11:46, 353.16s/it]

('Charlton Heston', 0.344845635801561)
('John Gielgud', 0.34395054343522674)
('Burt Lancaster', 0.3432575665174259)
('Henry Fonda', 0.340372001379352)
('John Carradine', 0.33982534652271745)
('James Stewart', 0.3387372876772099)
('David Niven', 0.3382220248321082)
('Robert Mitchum', 0.33639206310997954)
('Humphrey Bogart', 0.3358151203676688)
('Laurence Olivier', 0.3353687061165985)


Top 10 players by betweenness_centrality:


 75%|███████▌  | 3/4 [48:35<20:48, 1248.79s/it]

('Vincent Price', 0.010914519943179804)
('Burt Lancaster', 0.010773316674318037)
('John Carradine', 0.010629300889984141)
('Robert deNiro', 0.010469870030993679)
('Humphrey Bogart', 0.01038915996299399)
('Gene Hackman', 0.010043675685413013)
('John Gielgud', 0.009598596782090352)
('Jack Nicholson', 0.009224082251596292)
('Charlton Heston', 0.009194283586915035)
('James Stewart', 0.00819317097422135)


Top 10 players by eigenvector_centrality:


100%|██████████| 4/4 [48:36<00:00, 729.13s/it] 

('C.Aubrey Smith', 0.10527683317479046)
('John Carradine', 0.09890130374344182)
('James Stewart', 0.09305088872021564)
('Peter Lorre', 0.09255643718949176)
('John Gielgud', 0.0915831074918527)
('Basil Rathbone', 0.08981921063491403)
('Gary Cooper', 0.08938955258000875)
('David Niven', 0.08746695187553871)
('Andy Devine', 0.08728719728815929)
('Humphrey Bogart', 0.08503246285164423)




## Communities

In [7]:
communities = {node:cid+1 for cid,community in enumerate(nx.algorithms.community.k_clique_communities(G,59)) for node in community}
print('Largest community has size of {} nodes:'.format(len(communities)))
print_dict(communities, len(communities))
print('\n')

communities = {node:cid+1 for cid,community in enumerate(nx.algorithms.community.k_clique_communities(G,35)) for node in community}
print('Communities with at least 35 nodes:')
print_dict(communities, len(communities))

Largest community has size of 59 nodes:
('Sir Godrey Teale', 1)
('Sebastian Cabot', 1)
('C.Aubrey Smith', 1)
('Bill Travers', 1)
('Francesca Bertini', 1)
('Laurence Olivier', 1)
('Aldo Zollo', 1)
('Harry Hilliard', 1)
('Paul Hardwick', 1)
('Ibrahim Hamouda', 1)
('Henry Kolker', 1)
('Ralph Forbes', 1)
('Olivia Hussey', 1)
('Mervyn Johns', 1)
('Virginia Hammond', 1)
('Leslie Howard', 1)
('Mary Malone', 1)
('George A. Lessey', 1)
('Andy Devine', 1)
('Norma Shearer', 1)
('Francis X. Bushman', 1)
('Natasha Peryy', 1)
('td> Claire Danes<', 1)
('Rosemarie Dexter', 1)
('John Gielgud', 1)
('Norman Wooland', 1)
('Reginald Denny', 1)
('Theda Bara', 1)
('Esmeralda Ruspoli', 1)
('Florence Lawrence', 1)
('Maria Gasperini', 1)
('Lela Mourad', 1)
('Leonardo DiCaprio', 1)
('Beverly Bane', 1)
('Mario Caserini', 1)
('Nietta Zocchi', 1)
('Bruce Robinson', 1)
('Leonard Whiting', 1)
('Flora Robson', 1)
('Gustav Serena', 1)
('Michael York', 1)
('Roberto Bisacco', 1)
('Antionio Pierfrederici', 1)
('Conway Tea

## Kevin Bacon numbers

In [8]:
def kevin_bacon_numbers(graph):
    for a in graph.nodes():
        try:
            path = nx.shortest_path(graph,source=a,target='Kevin Bacon')
            graph.nodes[a]['kevin_bacon_number'] = int(len(path)/2)
        except nx.NetworkXNoPath:
            graph.nodes[a]['kevin_bacon_number'] = len(graph.nodes())
#         print('{0}: {1}'.format(a, graph.nodes[a]))
    return graph

print('Top {} actors with Kevin Bacon number:'.format(k))
G = kevin_bacon_numbers(G)
# print(G.nodes['Humphrey Bogart'])


#=== Top k actors by Kevin Bacon number (including infinite KB number - actors wign non-existing path to KB)
# Sort actors by Kevin Bacon number
kevin_bacon_desc = dict(sorted(dict(G.nodes(data=True)).items(), key=lambda item: item[1]['kevin_bacon_number'], reverse=True))
print_dict(kevin_bacon_desc)
print('\n')


#=== Top k actors by Kevin Bacon number (finite only)
kevin_bacon_decs_fin = {}

# Filter actors by finite Kevin Bacon number
to_add = [key for key in kevin_bacon_desc if kevin_bacon_desc[key]['kevin_bacon_number'] is not math.inf]
for key in to_add:
    kevin_bacon_decs_fin[key] = kevin_bacon_desc[key]
    
print('Top {} actors with finite Kevin Bacon number:'.format(k))
print_dict(kevin_bacon_decs_fin)
print('\n')


#=== Bottom k actors by Kevin Bacon number
print('Bottom {} actors with finite Kevin Bacon number:'.format(k))
kevin_bacon_asc_fin = dict(sorted(kevin_bacon_decs_fin.items(), key=lambda item: item[1]['kevin_bacon_number']))
print_dict(kevin_bacon_asc_fin)
print('\n')

#=== Average Kevin Bacon number (finite numbers only)
sum = 0
for item in kevin_bacon_asc_fin.items():
    sum += item[1]['kevin_bacon_number']
kv_avg = sum/len(kevin_bacon_asc_fin)
print('Average Kevin Bacon number: {}'.format(kv_avg))

Top 10 actors with Kevin Bacon number:
('Abel Gance', {'degree_centrality': 0.00018062496236979952, 'closeness_centrality': 0.00018062496236979952, 'betweenness_centrality': 0.0, 'eigenvector_centrality': 3.674385679978072e-21, 'kevin_bacon_number': 16610})
('Abel Salazar', {'degree_centrality': 6.020832078993317e-05, 'closeness_centrality': 6.020832078993317e-05, 'betweenness_centrality': 0.0, 'eigenvector_centrality': 2.2426670410022412e-25, 'kevin_bacon_number': 16610})
('Abishek Kapoor', {'degree_centrality': 6.020832078993317e-05, 'closeness_centrality': 6.020832078993317e-05, 'betweenness_centrality': 0.0, 'eigenvector_centrality': 2.2426670410022412e-25, 'kevin_bacon_number': 16610})
('Adolfas Mekas', {'degree_centrality': 0.00024083328315973267, 'closeness_centrality': 0.00024083328315973267, 'betweenness_centrality': 0.0, 'eigenvector_centrality': 8.354585770525938e-20, 'kevin_bacon_number': 16610})
('Adolph Gance', {'degree_centrality': 0.0, 'closeness_centrality': 0.0, 'betw

## Save into GEXF format

In [9]:
nx.write_gexf(G, './../results/actors_casts.gexf')

## Dataset reduction

In [10]:
movies_casts_cnt = casts['movie'].value_counts().to_dict()
print(len(movies_casts_cnt))
# print(movies_casts_cnt)

movies_reduced = list(k for (k, v) in movies_casts_cnt.items() if (v < 5))
print(len(movies_reduced))
# print(movies_reduced)

movies_by_actor_reduced_dict = {}

for key, val in movies_by_actor_dict.items():
    if lists_overlap(val, movies_reduced):
        movies_by_actor_reduced_dict[key] = val

print(len(movies_by_actor_reduced_dict))
# print(movies_by_actor_reduced_dict)

8632
4356
6090


## Construct reduced graph

In [11]:
# Create the graph
G_reduced = nx.Graph()

for actor in movies_by_actor_reduced_dict.keys():
    G_reduced.add_node(actor)

for actor1, movies1 in tqdm(movies_by_actor_reduced_dict.items(), total=len(G_reduced.nodes())):
    for actor2, movies2 in movies_by_actor_reduced_dict.items():
        if actor1 != actor2 and lists_overlap(movies1, movies2):
            G_reduced.add_edge(actor1, actor2)

100%|██████████| 6090/6090 [00:33<00:00, 183.43it/s]


## Reduced graph general statistics

In [12]:
general_statistics(G_reduced)

Number of nodes:  6090
Number of edges:  44412
Density:  0.0023953394112131462
Number of components:  669


## Reduced Centralities

In [13]:
G_reduced = compute_centralities(G_reduced, [nx.degree_centrality, nx.closeness_centrality, nx.betweenness_centrality, nx.eigenvector_centrality], k)

  0%|          | 0/4 [00:00<?, ?it/s]

Top 10 players by degree_centrality:
('Charlton Heston', 0.032846115946789295)
('Burt Lancaster', 0.03169650188865167)
('John Carradine', 0.03169650188865167)
('Henry Fonda', 0.031532271308917725)
('James Stewart', 0.031532271308917725)
('John Gielgud', 0.031039579569715883)
('Gary Cooper', 0.030382657250780095)
('Humphrey Bogart', 0.029397273772376418)
('Gene Hackman', 0.02890458203317458)
('C.Aubrey Smith', 0.0279191985547709)


Top 10 players by closeness_centrality:


 50%|█████     | 2/4 [00:36<00:36, 18.03s/it]

('Burt Lancaster', 0.310685667814704)
('Charlton Heston', 0.30940184274108956)
('John Gielgud', 0.30855183767861405)
('Robert Mitchum', 0.3041769589860226)
('Henry Fonda', 0.30395904983766037)
('Roddy McDowall', 0.30225087407871837)
('John Carradine', 0.3012969499061486)
('Gene Hackman', 0.3006327777602367)
('Martin Balsam', 0.30030178854282225)
('Laurence Olivier', 0.30027817434226056)


Top 10 players by betweenness_centrality:


 75%|███████▌  | 3/4 [03:48<01:30, 90.82s/it]

('Burt Lancaster', 0.011316000218762638)
('John Gielgud', 0.01011536998385631)
('Robert deNiro', 0.009845090840123924)
('Gene Hackman', 0.009550366517573454)
('Charlton Heston', 0.008972107908516226)
('Jack Nicholson', 0.008110236359800432)
('Ingrid Bergman', 0.007982081948154812)
('John Carradine', 0.007771133819277224)
('Max vonSydow', 0.007355403373209441)
('Sean Connery', 0.007291372602422415)


Top 10 players by eigenvector_centrality:


100%|██████████| 4/4 [03:49<00:00, 57.30s/it]

('John Carradine', 0.11467320567694805)
('Henry Fonda', 0.10901593912198233)
('James Stewart', 0.1043326997428101)
('C.Aubrey Smith', 0.1033950443827341)
('David Niven', 0.10139095210817224)
('Charlton Heston', 0.09939158317938407)
('Gary Cooper', 0.09491106954410376)
('John Gielgud', 0.09016101051755374)
('Peter Lorre', 0.0882240195382321)
('Humphrey Bogart', 0.08555892800714512)




## Reduced communities

In [14]:
communities = {node:cid+1 for cid,community in enumerate(nx.algorithms.community.k_clique_communities(G_reduced,20)) for node in community}
print('Communities with at least 20 nodes:')
print_dict(communities, len(communities))

for key, val in G_reduced.nodes().items():
    if key in communities:
        G_reduced.nodes[key]['community'] = communities[key]
    else:
        G_reduced.nodes[key]['community'] = 0

Communities with at least 20 nodes:
('Vincent dOnofrio', 1)
('Andie McDowell', 1)
('Peter Gallagher', 1)
('Nick Nolte', 1)
('Burt Reynolds', 1)
('Fred Ward', 1)
('Tim Robbins', 1)
('Cher', 1)
('Anjelica Huston', 1)
('Buck Henry', 1)
('Whoopie Goldberg', 1)
('Bruce Willis', 1)
('Greta Scacchi', 1)
('Dina Merrill', 1)
('Dustin Hoffman', 1)
('Jack Lemmon', 1)
('Dean Stockwell', 1)
('Julia Roberts', 1)
('Lyle Lovett', 1)
('Susan Sarandon', 1)
('Robert Morley', 3)
('Victor McLaglen', 2)
('Tim McCoy', 2)
('Basil Sidney', 2)
('Cesar Romero', 2)
('Gilbert Roland', 2)
('Ronald Colman', 2)
('Fernandel', 2)
('Charles Boyer', 2)
('Marlene Dietrich', 2)
('Glynis Johns', 2)
('Evelyn Keyes', 2)
('Joe E. Brown', 2)
('Frank Sinatra', 2)
('John Gielgud', 3)
('Red Skelton', 2)
('Reginald Denny', 2)
('David Niven', 2)
('John Mills', 2)
('John Carradine', 2)
('Robert Newton', 2)
('Buster Keaton', 2)
('Trevor Howard', 2)
('Peter Lorre', 2)
('George Raft', 2)
('David Dukes', 3)
('Jeremy Kemp', 3)
('Hart Boch

## Reduced Kevin Bacon numbers

In [15]:
G_reduced = kevin_bacon_numbers(G_reduced)
print_dict(G_reduced.nodes())

('58 Plymouth Fury', {'degree_centrality': 0.00016423057973394647, 'closeness_centrality': 0.1791862204077954, 'betweenness_centrality': 0.0, 'eigenvector_centrality': 6.383975775530849e-05, 'community': 0, 'kevin_bacon_number': 2})
('Aaron Schwartz', {'degree_centrality': 0.00032846115946789294, 'closeness_centrality': 0.1919106213242802, 'betweenness_centrality': 0.0, 'eigenvector_centrality': 0.00011137954845097149, 'community': 0, 'kevin_bacon_number': 2})
('Abel Gance', {'degree_centrality': 0.0004926917392018395, 'closeness_centrality': 0.0004926917392018393, 'betweenness_centrality': 0.0, 'eigenvector_centrality': 1.9742043101430193e-21, 'community': 0, 'kevin_bacon_number': 6090})
('Abel Salazar', {'degree_centrality': 0.00016423057973394647, 'closeness_centrality': 0.00016423057973394647, 'betweenness_centrality': 0.0, 'eigenvector_centrality': 3.012396713475066e-26, 'community': 0, 'kevin_bacon_number': 6090})
('Abishek Kapoor', {'degree_centrality': 0.00016423057973394647, '

## Save into GEFX format

In [16]:
nx.write_gexf(G_reduced, './../results/actors_casts_reduced.gexf')

## Testing attributes

In [17]:
print(G_reduced.nodes['Matthew Settle'])

{'degree_centrality': 0.0004926917392018395, 'closeness_centrality': 0.0004926917392018393, 'betweenness_centrality': 0.0, 'eigenvector_centrality': 1.9742043101430193e-21, 'community': 0, 'kevin_bacon_number': 6090}
